# Aspects (plus) avancés pour la bioinformatique: obtenir une séquence directement de GenBank au NCBI
De temps à autre, on a besoin d'une séquence particulière pour y mettre notre grain de sel: clonage, séquences orthologues, etc. Il est possible d'aller chercher directement ces séquences (de quelque nature que ce soit) via la méthode des URL spécialement écrits. Pas mal toutes les informations du NCBI sont disponibles en utilisant les outils E-Utilities, spécifiquemment la fonction `efetch`.
Pour que cette méthode fonctionne, on doit obligatoirement spécifié certains paramètres:

 - `db`: la base de données du NCBI à utiliser. La liste compléte des bases de données est disponibles ici: [https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi](https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi)
 - `id`: indentificateur de l'item à récupérer dans la base de données spécifiée par db;
 - `rettype`: le format à utiliser pour les données qui vous reviendront;
 - `retmode`: le type de données qui reviendra.

Dans les exemples qui suivront, nous allons rechercher des séquences par GeneID et par position chromosomique.

### Extraction de séquences via identificateur unique

In [1]:
import requests

efetchURL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?"
# ARNm de l'isoforme A de la CTTN
cttn_rna = "NM_005231.4"
# Seq. correspondante en acides aminés
cttn_aa = "NP_005222.2"
rettype = "fasta"
retmode = "text"

# Preparation de la requete
headers = {'content-type': 'application/x-www-form-urlencoded'}
eutils_url_rna = efetchURL+"db=nuccore&id="+cttn_rna+"&rettype="+rettype+"&retmode="+retmode
print(eutils_url_rna)

# Envoyez la requete
res = requests.get(eutils_url_rna,headers=headers)

# Le flux de données est en binaire; il faut donc le convertir en texte on the fly
seq_nuc = res.content.decode('UTF-8')
print(seq_nuc)

eutils_url_aa = efetchURL+"db=protein&id="+cttn_aa+"&rettype="+rettype+"&retmode="+retmode

res = requests.get(eutils_url_aa,headers=headers)

# Le flux de données est en binaire; il faut donc le convertir en texte on the fly
seq_aa = res.content.decode('UTF-8')
print(seq_aa)

# Après, faite ce qui doit être fait avec votre séquence :-)

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id=NM_005231.4&rettype=fasta&retmode=text
>NM_005231.4 Homo sapiens cortactin (CTTN), transcript variant 1, mRNA
GGAACCGGAAGTAGAGCCTGGTGCCTGGGAGCGGCTGGCGCGGCGGAATCCAGGGCCGACCCGGGCCGGA
CCGACCCCAGGCGGCGACGGAATCAGTCCCCAATGCCTGGAAATTCCTCATTGGATTACTGTGTTTTAAA
CAGAATTTCGTGAACAGCCTTTTATCTCCAAGCGGAAAGAAAGATGTGGAAAGCTTCAGCAGGCCACGCT
GTGTCCATCGCCCAGGATGACGCGGGGGCCGATGACTGGGAGACCGACCCTGATTTTGTGAATGATGTGA
GTGAGAAGGAGCAAAGATGGGGTGCCAAGACGGTGCAGGGCTCCGGGCACCAGGAGCATATCAACATACA
CAAGCTGAGGGAGAATGTCTTTCAAGAGCATCAGACCCTTAAGGAGAAGGAACTTGAAACAGGACCAAAA
GCTTCCCATGGCTATGGAGGGAAATTTGGTGTGGAACAAGACCGAATGGATAAGTCAGCTGTCGGCCACG
AATATCAGTCGAAACTTTCCAAGCACTGCTCGCAGGTGGACTCGGTCCGTGGCTTCGGAGGCAAGTTTGG
TGTCCAGATGGACAGAGTTGATCAGTCTGCTGTAGGCTTTGAATACCAGGGGAAGACTGAGAAGCATGCC
TCCCAGAAAGACTACTCCAGTGGTTTTGGCGGCAAGTATGGCGTGCAGGCCGACCGAGTAGACAAGAGCG
CGGTGGGCTTCGACTACCAGGGCAAGACGGAGAAGCACGAGTCACAGAGAGATTACTCCAAAGGTTTCGG
CGGCAAATACGGTATCGACAAGGACAAAGTGGATAAG

### Extraction d'une portion de séquence provenant d'un identificateur unique

In [1]:
import requests

efetchURL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?"
# Sequence du chr 11, ou se trouve CTTN
cttn_gene = "NC_000011.10"
# Positions des début et fin de l'exon 1
cttn_x1_start = 70398529
cttn_x1_stop = 70436575
rettype = "fasta"
retmode = "text"

# Preparation de la requete
headers = {'content-type': 'application/x-www-form-urlencoded'}
params = {
    "db" : "nuccore",
    "id" : cttn_gene,
    "seq_start" : cttn_x1_start,
    "seq_stopt" : cttn_x1_stop,
    "rettype" : rettype,
    "retmode" : retmode
}
# Envoyez la requete
res = requests.get(efetchURL,params=params,headers=headers)

# Le flux de données est en binaire; il faut donc le convertir en texte on the fly
seq_x1 = res.content.decode('UTF-8')
# On reçoit une très longue chaine mélangeant commentaires et sequences MAIS 
# contenant le caractères '\n' :-)
allLines = seq_x1.split("\n")
print(allLines[0:3])

['>NC_000011.10:70398529-135086622 Homo sapiens chromosome 11, GRCh38.p14 Primary Assembly', 'GGAACCGGAAGTAGAGCCTGGTGCCTGGGAGCGGCTGGCGCGGCGGAATCCAGGGCCGACCCGGGCCGGA', 'CCGACCCCAGGCGGCGGTGAGCGAGCGCGGCGTCCGCCCGGGGTGCAGGCCGGGCTCTGCTTCTTCCCTC']
